In [194]:
year_data = 2025

In [195]:
import requests
import pandas as pd

In [ ]:
ergast_base_url = "https://api.jolpi.ca/ergast/f1/"
# https://github.com/jolpica/jolpica-f1/blob/main/docs/README.md

In [ ]:
def get_drivers_df(year):
    url = f"{ergast_base_url}/{year}/drivers/"
    response = requests.get(url)
    drivers_list = response.json()['MRData']['DriverTable']['Drivers']
    return pd.json_normalize(drivers_list)

In [ ]:
def get_constructors_df(year):
    url = f"{ergast_base_url}/{year}/constructors/"
    response = requests.get(url)
    constructors_list = response.json()['MRData']['ConstructorTable']['Constructors']
    return pd.json_normalize(constructors_list)

In [ ]:
def get_races_calendar_df(year):
    url = f"{ergast_base_url}/{year}"
    response = requests.get(url)
    races_list = response.json()['MRData']['RaceTable']['Races']
    return pd.json_normalize(races_list)

In [ ]:
def get_races_results_df(year, year_races_calendar_df):
    # Start season races pd df
    races_df = pd.DataFrame()

    # Get season length
    season_length = len(year_races_calendar_df.index)

    # For each race
    for round_index in range(1,season_length+1):
        try:
            url = f"{ergast_base_url}/{year}/{round_index}/results"
            response = requests.get(url)

            # Get race result into a pd df
            race_result_df = pd.json_normalize(response.json()['MRData']['RaceTable']['Races'][0]['Results'])

            # Add circuit id, season year, round id
            race_row = year_races_calendar_df.loc[year_races_calendar_df['round'].astype(int) == round_index]
            race_row = race_row.to_dict(orient='records')[0]

            race_result_df['raceRoundId'] = race_row['round']
            race_result_df['seasonYear'], race_result_df['circuitId'] = race_row['season'], race_row['Circuit.circuitId']

            # Add race result df to season races df
            races_df = pd.concat([races_df, race_result_df], ignore_index=True)
        except:
            # If no more races
            break 

    # Keep these columns 
    races_df = races_df[['number','position','positionText','points','grid','laps','status','position','Driver.permanentNumber','Driver.code'
        ,'Constructor.constructorId','Time.millis','Time.time','FastestLap.rank','FastestLap.lap','FastestLap.Time.time','raceRoundId','seasonYear','circuitId']]
    
    return races_df

In [ ]:
if __name__ == "__main__":
    year_drivers_df = get_drivers_df(year_data)
    year_constructors_df = get_constructors_df(year_data)
    year_races_calendar_df = get_races_calendar_df(year_data)
    year_data_df = get_races_results_df(year_data, year_races_calendar_df)

In [ ]:
#print(year_drivers_df.to_markdown())
#print(year_constructors_df.to_markdown())
#print(year_races_calendar_df.to_markdown())

In [199]:
#print(year_data_df.to_markdown())